In [1]:
# import of the needed packages
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import math
import re
import glob
import os

# Season based Preprocessing

In [2]:
def append_actual_matchday(df_bundesliga_wo_matchday: pd.DataFrame):
    # calculates home many games a has already played until the matchday in a season
    # can be differed between home and away games and between the teams

    games_played_this_season_ht_home = []
    games_played_this_season_ht_away = []
    games_played_this_season_at_home = []
    games_played_this_season_at_away = []

    for index, rows in df_bundesliga_wo_matchday.iterrows():
        home_team_temp = rows.HomeTeam
        away_team_temp = rows.AwayTeam
        games_played_all_teams_home = df_bundesliga_wo_matchday.iloc[:index].HomeTeam.value_counts()
        games_played_all_teams_away = df_bundesliga_wo_matchday.iloc[:index].AwayTeam.value_counts()
        games_played_this_season_ht_home.append(0 if home_team_temp not in games_played_all_teams_home else games_played_all_teams_home[home_team_temp])
        games_played_this_season_ht_away.append(0 if home_team_temp not in games_played_all_teams_away else games_played_all_teams_away[home_team_temp])
        games_played_this_season_at_home.append(0 if away_team_temp not in games_played_all_teams_home else games_played_all_teams_home[away_team_temp])
        games_played_this_season_at_away.append(0 if away_team_temp not in games_played_all_teams_away else games_played_all_teams_away[away_team_temp])

    df_bundesliga_wo_matchday.loc[:,"GPHTH"] = games_played_this_season_ht_home
    df_bundesliga_wo_matchday.loc[:,"GPHTA"] = games_played_this_season_ht_away
    df_bundesliga_wo_matchday.loc[:,"GPATH"] = games_played_this_season_at_home
    df_bundesliga_wo_matchday.loc[:,"GPATA"] = games_played_this_season_at_away

    # calculate total games played
    df_bundesliga_wo_matchday.loc[:,"GPHT"] = df_bundesliga_wo_matchday.loc[:,"GPHTH"] + df_bundesliga_wo_matchday.loc[:,"GPHTA"]
    df_bundesliga_wo_matchday.loc[:,"GPAT"] = df_bundesliga_wo_matchday.loc[:,"GPATH"] + df_bundesliga_wo_matchday.loc[:,"GPATA"]
    return df_bundesliga_wo_matchday

In [3]:
def get_form_of_last_matches(df_bundesliga_wo_form: pd.DataFrame):
    # calculate how many points a team has gained out of the last 3, 5 and 10 matches:
    list_number_of_matches = [3, 5, 10, 34]
    last_match_points = []

    for index, rows in df_bundesliga_wo_form.iterrows():
        last_match_points_temp = []
        for i in list_number_of_matches:
            home_team_temp = rows.HomeTeam
            away_team_temp = rows.AwayTeam
            teams_points_temp = []
            for team in [home_team_temp, away_team_temp]:
                if rows.GPHT >= i or i == 34:
                    df_last_games_ht = df_bundesliga_wo_form[(df_bundesliga_wo_form.GPHT < rows.GPHT)]
                    if i != 34:
                        df_last_games_ht = df_last_games_ht[(df_last_games_ht.GPHT >= rows.GPHT - i)]
                    # calculate all points at home
                    df_last_games_specific_ht = df_last_games_ht[(df_last_games_ht.HomeTeam==team)].FTR.value_counts()
                    points = 0
                    if 'H' in df_last_games_specific_ht:
                        points += 3 * df_last_games_specific_ht['H']
                    if 'D' in df_last_games_specific_ht:
                        points += df_last_games_specific_ht['D']
                    # calculate all points away
                    df_last_games_specific_ht = df_last_games_ht[(df_last_games_ht.AwayTeam==team)].FTR.value_counts()
                    if 'A' in df_last_games_specific_ht:
                        points += 3 * df_last_games_specific_ht['A']
                    if 'D' in df_last_games_specific_ht:
                        points += df_last_games_specific_ht['D']
                    teams_points_temp.append(points)
                else:
                    teams_points_temp = [None, None]
            last_match_points_temp.extend(teams_points_temp)
        last_match_points.append(last_match_points_temp)
    return last_match_points

In [4]:
def add_market_values(season: str):
    # add market values
    # preprocess csv data with market values
    market_values_bl = pd.read_csv(f"market_values_df\\market_values_bl_{season}.csv", delimiter=";")
    # map clubs due to different naming in different files
    map_team_names_dict = {
        '1. FC Köln': 'FC Koln',
        '1.FC Union Berlin': 'Union Berlin',
        '1.FSV Mainz 05': 'Mainz',
        'Arminia Bielefeld': 'Bielefeld',
        'Bayer 04 Leverkusen': 'Leverkusen',
        'Bayern Munich': 'Bayern Munich',
        'Borussia Dortmund': 'Dortmund',
        'Borussia Mönchengladbach': "M'gladbach",
        'Eintracht Frankfurt': 'Ein Frankfurt',
        'FC Augsburg': 'Augsburg',
        'Hertha BSC': 'Hertha',
        'RB Leipzig': 'RB Leipzig',
        'SC Freiburg': 'Freiburg',
        'SpVgg Greuther Fürth': 'Greuther Furth',
        'TSG 1899 Hoffenheim': 'Hoffenheim',
        'VfB Stuttgart': 'Stuttgart',
        'VfL Bochum': 'Bochum',
        'VfL Wolfsburg': 'Wolfsburg',
        'SV Werder Bremen': "Werder Bremen", 
        'FC Schalke 04': "Schalke 04",
        'SC Paderborn 07': 'Paderborn',
        'Fortuna Düsseldorf': 'Fortuna Dusseldorf',
        '1.FC Nuremberg': 'Nurnberg',
        'Hannover 96': 'Hannover',
        'Hamburger SV': 'Hamburg',
        'SV Darmstadt 98': 'Darmstadt',
        'FC Ingolstadt 04': 'Ingolstadt',
        'Eintracht Braunschweig': 'Braunschweig',
        '1.FC Kaiserslautern': 'Kaiserslautern',
        'FC St. Pauli': 'St Pauli',
        'FC Energie Cottbus': 'Cottbus',
        'Karlsruher SC': 'Karlsruhe',
        'FC Hansa Rostock': 'Hansa Rostock',
        'MSV Duisburg': 'Duisburg',
        'Alemannia Aachen': 'Aachen'
    }
    market_values_bl.club = market_values_bl.club.replace(map_team_names_dict)
    if int(season) <= 10:
        market_value_column_name = f"Total market value"
    elif season == "14":
        market_value_column_name = f"Value Jul 10, 20{season}"
    else:
        market_value_column_name = f"Value Sep 15, 20{season}"
    market_values_bl = market_values_bl.dropna(subset=['club'])
    market_values_bl[market_value_column_name] = market_values_bl[market_value_column_name].apply(lambda x: float(re.search(r"\d+\.\d{2}", x).group()))
    return market_values_bl, market_value_column_name

In [5]:
def concat_mw_to_df(market_values_bl: pd.DataFrame, df_bundesliga_with_form: pd.DataFrame, market_value_column_name: str):
    # add market values to df
    market_value_HT = []
    market_value_AT = []
    for index, rows in df_bundesliga_with_form.iterrows():
        mw_ht = market_values_bl[market_values_bl.club==rows.HomeTeam]
        market_value_HT.append(float(mw_ht[market_value_column_name]))
        mw_at = market_values_bl[market_values_bl.club==rows.AwayTeam]
        market_value_AT.append(float(mw_at[market_value_column_name]))

    df_bundesliga_with_form["MarketValueHT"] = market_value_HT
    df_bundesliga_with_form["MarketValueAT"] = market_value_AT
    df_bundesliga_with_form["MarketValueDiff"] = df_bundesliga_with_form.loc[:, "MarketValueHT"] - df_bundesliga_with_form.loc[:, "MarketValueAT"]
    df_bundesliga_with_form["MarketValueQuot"] = df_bundesliga_with_form.loc[:, "MarketValueHT"]/df_bundesliga_with_form.loc[:, "MarketValueAT"]
    return df_bundesliga_with_form

In [6]:
# make season based processing over all seasons

def run_all():
    for i in range(1, 16):
        current_bl_dataframe = pd.read_csv(f"all_saison_bl_game_data\\D1 ({i}).csv")
        season = 22 - i
        if season < 10:
            season = f"0{season}"
        else:
            season = str(season)
        current_bl_dataframe_with_matchdays = append_actual_matchday(current_bl_dataframe)
        last_match_points = get_form_of_last_matches(current_bl_dataframe)
        df_bundesliga_with_form = pd.concat([current_bl_dataframe_with_matchdays, pd.DataFrame(last_match_points, columns=["PointsLast3MatchesHT", "PointsLast3MatchesAT", "PointsLast5MatchesHT", "PointsLast5MatchesAT", "PointsLast10MatchesHT", "PointsLast10MatchesAT","PointsLastAllMatchesHT", "PointsLastAllMatchesAT"])], axis=1)
        market_values_bl, market_value_column_name = add_market_values(season)
        df_bundesliga_with_mw = concat_mw_to_df(market_values_bl, df_bundesliga_with_form, market_value_column_name)
        # save modified bl game information of one season in new file
        df_bundesliga_with_mw.to_csv(f'Data_BL_modified//bundesliga_data_modified_season_{season}.csv')
run_all()

# Concat all Season Data to one File

In [7]:
# concat all files over several seasons to one dataframe and saves as csv

def concat_all_season_based_files():
    path = "Data_BL_modified//"
    all_files = glob.glob(os.path.join(path, "*.csv"))

    all_df = []
    for f in all_files:
        df = pd.read_csv(f, sep=',', on_bad_lines="skip")
        df['file'] = f.split('\\')[-1]
        all_df.append(df)
        
    merged_df = pd.concat(all_df, ignore_index=True, sort=False)
    merged_df = merged_df.dropna(axis=1, how="all")
    merged_df.to_csv("cumulated_data_bl_final_version.csv")
concat_all_season_based_files()

# Preprocessing not season based

In [8]:
# function to load the raw data and add the necessary columns for our calculated features
def load_data(file):
    # load the raw data
    data = pd.read_csv(file, delimiter=',')
    # adding the columns for the calculated features
    data['HomeElo'] = 0
    data['AwayElo'] = 0
    data['HomeAttack'] = 0
    data['HomeDefend'] = 0
    data['AwayAttack'] = 0
    data['AwayDefend'] = 0
    data['HomeEloOld'] = 0
    data['AwayEloOld'] = 0
    data['HomeAttackOld'] = 0
    data['HomeDefendOld'] = 0
    data['AwayAttackOld'] = 0
    data['AwayDefendOld'] = 0
    data['DiffEloOld'] = 0
    data['DiffAttackOld'] = 0
    data['DiffDefendOld'] = 0
    data['DiffElo'] = 0
    data['DiffAttack'] = 0
    data['DiffDefend'] = 0
    data['QuotEloOld'] = 0
    data['QuotAttackOld'] = 0
    data['QuotDefendOld'] = 0
    data['QuotElo'] = 0
    data['QuotAttack'] = 0
    data['QuotDefend'] = 0
    
    return data

In [9]:
# function to get the names of all teams in the data frame
def get_all_teams(data):
    teams = []
    # loop to store the team names into a list
    for x in data['HomeTeam']:
        if x not in teams:
            teams.append(x)
            
    return teams

In [10]:
# function to create a dictionary to store the home/away elo and attack/defend values for each team
def create_elo_dictionary(teams):

    teams_elo_avstaerke = {}
    # loop for generating the dictionary to store the calculated values
    # setting standard values for home/away elo, attack/defend values to 
    # avoid problems with divering values for the teams that are promoted to the BL and would start with 0
    for x in teams:
        team_stats = {x: [1000, 1000, 1.34, -1.34]}
        teams_elo_avstaerke.update(team_stats)
        
    return teams_elo_avstaerke

In [11]:
# function to calculate the difference between the elos
def diffelo(homeelo, awayelo):
    diffelo = homeelo - awayelo
    
    return diffelo

In [12]:
# function to calucalate the elo gain for the teams in the match
def elogain_2(z, data, hometeam, awayteam, teams_elo_avstaerke):
    # loading the elo values for the home and the away team from the dictionary
    homeelo = teams_elo_avstaerke[hometeam][0]
    awayelo = teams_elo_avstaerke[awayteam][0]

    # getting the true outcome of the game from the data frame
    trueoutcome = outcometrue_2(z, data)

    # calculating the possible outcome and a margin of victory based on the elo values of the teams
    expectedoutcome = outcomeexpected_2(z, homeelo, awayelo)
    ofvictorymargin = marginofvictory_2(z, data, homeelo, awayelo)

    # calculating the elo gain for both teams and adding it to their previous value while taking the marginofvictory into account
    elogain = 20 * (trueoutcome - expectedoutcome) * ofvictorymargin
    elohome = homeelo + elogain
    eloaway = awayelo - elogain

    # updating the dictionary with the newly calculated elo values
    teams_elo_avstaerke[hometeam][0] = elohome
    teams_elo_avstaerke[awayteam][1] = eloaway

    return elohome, eloaway, homeelo, awayelo

In [13]:
# function for calculating the margin of victory
# the margin of victory is used to incorporate the difference of the teams
# as its easier for good teams to score against bad teams
# scoring against good teams is honored, scoring against bad teams gets downgraded 
def marginofvictory_2(z, data, homeelo, awayelo):
    # getting the elo and the goal difference between the teams
    elodiff = diffelo(homeelo, awayelo)
    diffgoal = data.FTHG[z] - data.FTAG[z]

    if diffgoal <= 1:
        marginofvictory = 1
    else:
        # calculating the margin of victory to honor scoring against good teams
        marginofvictory = (math.log2(1.7 * diffgoal) * 2) / (2 +
                                                             0.001 * elodiff)
    return marginofvictory

In [14]:
# function to get the outcome of the game and transferring it into numerical values
def outcometrue_2(z, data):
    if data.FTR[z] == "H":
        outcometrue = 1
    elif data.FTR[z] == "D":
        outcometrue = 0.5
    else:
        outcometrue = 0
    return outcometrue

In [15]:
# function to calculate a possible outcome for the game
def outcomeexpected_2(z, homeelo, awayelo):
    elodiff = diffelo(homeelo, awayelo)
    # calculating the game outcome based on the elo difference
    outcomeexpected = 1 / (1 + 10**(-elodiff / 400))
    return outcomeexpected

In [16]:
# function to calculate the attack value for the home team
def home_attack(z, data, alpha, ratio, hometeam, awayteam,
                teams_elo_avstaerke):
    # get the match goals and the old attack and defend values
    goalsshot = data.FTHG[z]
    attack_old = teams_elo_avstaerke[hometeam][2]
    otherteamdefend = teams_elo_avstaerke[awayteam][3]
    
    # calculate the new attack value
    homeattack = attack_old + ((goalsshot - attack_old) * ratio +
                               (goalsshot + otherteamdefend) *
                               (1 - ratio)) * alpha
    # update the attack value in the dictionary
    teams_elo_avstaerke[hometeam][2] = homeattack
    return homeattack, attack_old

In [17]:
# function to calculate the defend value for the home team
def home_defend(z, alpha, ratio, data, hometeam, awayteam,
                teams_elo_avstaerke):
    # get the match goals and the old attack and defend values
    goalsgot = data.FTAG[z]
    defend_old = teams_elo_avstaerke[hometeam][3]
    otherteamattack = teams_elo_avstaerke[awayteam][2]
    # calculate the new defend value
    homedefend = defend_old - ((goalsgot + defend_old) * ratio +
                               (goalsgot - otherteamattack) *
                               (1 - ratio)) * alpha
    # update the defend value in the dictionary
    teams_elo_avstaerke[hometeam][3] = homedefend
    return homedefend, defend_old

In [18]:
# function to calculate the attack value for the away team
def away_attack(z, alpha, ratio, data, hometeam, awayteam, teams_elo_avstaerke):
    # get the match goals and the old attack and defend values
    goalsshot = data.FTAG[z]
    attack_old = teams_elo_avstaerke[awayteam][2]
    otherteamdefend = teams_elo_avstaerke[hometeam][3]
    # calculate the new attack value
    awayattack = attack_old + ((goalsshot - attack_old) * ratio +
                               (goalsshot + otherteamdefend) *
                               (1 - ratio)) * alpha
    # update the attack value in the dictionary
    teams_elo_avstaerke[awayteam][2] = awayattack
    return awayattack, attack_old

In [19]:
# function to calculate the defend value for the away team
def away_defend(z, alpha, ratio, data, hometeam, awayteam, teams_elo_avstaerke):
    # get the match goals and the old attack and defend values
    goalsgot = data.FTHG[z]
    defend_old = teams_elo_avstaerke[awayteam][3]
    otherteamattack = teams_elo_avstaerke[hometeam][2]
    # calculate the new defend value
    awaydefend = defend_old - ((goalsgot + defend_old) * ratio +
                               (goalsgot - otherteamattack) *
                               (1 - ratio)) * alpha
    # update the attack value in the dictionary
    teams_elo_avstaerke[awayteam][3] = awaydefend
    return awaydefend, defend_old

In [20]:
# set initial value for elo, attack and defend to dataframe

def add_new_columns1(data):
    data['HomeElo'] = 0
    data['AwayElo'] = 0
    data['HomeAttack'] = 0
    data['HomeDefend'] = 0
    data['AwayAttack'] = 0
    data['AwayDefend'] = 0
    data['HomeEloOld'] = 0
    data['AwayEloOld'] = 0
    data['HomeAttackOld'] = 0
    data['HomeDefendOld'] = 0
    data['AwayAttackOld'] = 0
    data['AwayDefendOld'] = 0

In [21]:
# set initial value for elo, attack and defend difference and quotient to dataframe

def add_new_columns2(data):
    data['DiffEloOld'] = 0
    data['DiffAttackOld'] = 0
    data['DiffDefendOld'] = 0
    data['DiffElo'] = 0
    data['DiffAttack'] = 0
    data['DiffDefend'] = 0
    data['QuotEloOld'] = 0
    data['QuotAttackOld'] = 0
    data['QuotDefendOld'] = 0
    data['QuotElo'] = 0
    data['QuotAttack'] = 0
    data['QuotDefend'] = 0

In [22]:
# function to use all functions to calculate the elos and the attack/defend values for a game
def calculate_elo_attack_defense(data, teams_elo_avstaerke):
    z = 0
    # setting the values for alpha and ratio
    # explanation for what we use them!!!
    alpha = 0.5
    ratio = 0.75
    # loop to calculate the values for the whole data frame
    while z < len(data):
        # getting the home and away team for the game
        hometeam = data.HomeTeam[z]
        awayteam = data.AwayTeam[z]

        try:
            # calling the function to calculate the elo gains
            homeelo, awayelo, homeeloold, awayeloold = elogain_2(
            z, data, hometeam, awayteam, teams_elo_avstaerke)
            # calling the function to calculate the attack value for the home team
            homeattack1, homeattackold = home_attack(z, data, alpha, ratio,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            # calling the function to calculate the defend value for the home team
            homedefend1, homedefendold = home_defend(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            # calling the function to calculate the attack value for the away team
            awayattack1, awayattackold = away_attack(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
            # calling the function to calculate the defend value for the away team
            awaydefend1, awaydefendold = away_defend(z, alpha, ratio, data,
                                                     hometeam, awayteam,
                                                     teams_elo_avstaerke)
        except:
            print(f"Can't find data for team: {hometeam, awayteam}")
            break
        # adding the calucalted values to the data frame
        data['HomeElo'][z] = homeelo
        data['AwayElo'][z] = awayelo
        data['HomeAttack'][z] = homeattack1
        data['HomeDefend'][z] = homedefend1
        data['AwayAttack'][z] = awayattack1
        data['AwayDefend'][z] = awaydefend1
        data['HomeEloOld'][z] = homeeloold
        data['AwayEloOld'][z] = awayeloold
        data['HomeAttackOld'][z] = homeattackold
        data['HomeDefendOld'][z] = homedefendold
        data['AwayAttackOld'][z] = awayattackold
        data['AwayDefendOld'][z] = awaydefendold
        z+=1
    return data
    # return homeelo, awayelo, homeeloold, awayeloold, homeattack1, homeattackold, homedefend1, homedefendold, awayattack1, awayattackold, awaydefend1, awaydefendold


In [23]:
# adds elo, defend and attack to df for home and away team

def add_elo_attack_defense(homeelo, awayelo, homeeloold, awayeloold, homeattack1, homeattackold, homedefend1, homedefendold, awayattack1, awayattackold, awaydefend1, awaydefendold):
    data['HomeElo'][z] = homeelo
    data['AwayElo'][z] = awayelo
    data['HomeAttack'][z] = homeattack1
    data['HomeDefend'][z] = homedefend1
    data['AwayAttack'][z] = awayattack1
    data['AwayDefend'][z] = awaydefend1
    data['HomeEloOld'][z] = homeeloold
    data['AwayEloOld'][z] = awayeloold
    data['HomeAttackOld'][z] = homeattackold
    data['HomeDefendOld'][z] = homedefendold
    data['AwayAttackOld'][z] = awayattackold
    data['AwayDefendOld'][z] = awaydefendold


In [24]:
# function for adding the difference values for the elo, attack and defend values
def add_differences(data):
    data['DiffEloOld'] = data['HomeEloOld'] - data['AwayEloOld']
    data['DiffAttackOld'] = data['HomeAttackOld'] - data['AwayAttackOld']
    data['DiffDefendOld'] = data['HomeDefendOld'] - data['AwayDefendOld']
    data['DiffElo'] = data['HomeElo'] - data['AwayElo']
    data['DiffAttack'] = data['HomeAttack'] - data['AwayAttack']
    data['DiffDefend'] = data['HomeDefend'] - data['AwayDefend']
    return data

In [25]:
# function for adding the quotient values for the elo, attack and defend values
def add_qoutient(data):
    data['QuotEloOld'] = (data['HomeEloOld']) / (data['AwayEloOld'])
    data['QuotAttackOld'] = (data['HomeAttackOld']) / (data['AwayAttackOld'])
    data['QuotDefendOld'] = (data['HomeDefendOld']) / (data['AwayDefendOld'])
    data['QuotElo'] = (data['HomeElo']) / (data['AwayElo'])
    data['QuotAttack'] = (data['HomeAttack']) / (data['AwayAttack'])
    data['QuotDefend'] = (data['HomeDefend']) / (data['AwayDefend'])
    return data

In [26]:
# function to get the direct diffenrence betwenn the teams of a game
def direct_comparison(data):
    # get direct comparison of the two teams

    direct_comparison_hg = []
    direct_comparison_ag = []

    for counter, rows in data.iterrows():
        data_games_played_until_now = data.iloc[:counter]
        direct_comparison_teams_temp = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.HomeTeam)&(data_games_played_until_now.AwayTeam == rows.AwayTeam)|
        (data_games_played_until_now.HomeTeam == rows.AwayTeam)&(data_games_played_until_now.AwayTeam == rows.HomeTeam)]
        if len(direct_comparison_teams_temp) >= 3:
            if len(direct_comparison_teams_temp) >= 5:
                # take only the last 5 matches
                direct_comparison_teams_temp = direct_comparison_teams_temp.iloc[-5:]
                #direct_comparison_teams_temp.iloc[-5:].value_counts(["HomeTeam", "FTAG"])
                number_of_games = 5
            else:
                number_of_games = len(direct_comparison_teams_temp)

            # get average number of goals, that the hometeam has scored in the last 5 direct matches
            home_goals = (direct_comparison_teams_temp[direct_comparison_teams_temp.HomeTeam == rows.HomeTeam].FTHG.sum() + \
                direct_comparison_teams_temp[direct_comparison_teams_temp.AwayTeam == rows.HomeTeam].FTAG.sum()) / number_of_games

            # get average number of goals, that the awayteam has scored in the last 5 direct matches
            away_goals = (direct_comparison_teams_temp[direct_comparison_teams_temp.HomeTeam == rows.AwayTeam].FTHG.sum() + \
                direct_comparison_teams_temp[direct_comparison_teams_temp.AwayTeam == rows.AwayTeam].FTAG.sum()) / number_of_games

        else:
            # if the direct comparison is not possible yet (because both teams haven't played against each other) the scored goals of the last 10 matches would be measured to avoid null values
            goals_scored_ht = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.HomeTeam)|(data_games_played_until_now.AwayTeam == rows.HomeTeam)].iloc[-10:]
            goals_scored_at = data_games_played_until_now[(data_games_played_until_now.HomeTeam == rows.AwayTeam)|(data_games_played_until_now.AwayTeam == rows.AwayTeam)].iloc[-10:]

            if len(goals_scored_ht) >= 10:
                home_goals = (goals_scored_ht[goals_scored_ht.HomeTeam == rows.HomeTeam].FTHG.sum() + \
                    goals_scored_ht[goals_scored_ht.AwayTeam == rows.HomeTeam].FTAG.sum()) / 10
                away_goals = (goals_scored_at[goals_scored_at.HomeTeam == rows.AwayTeam].FTHG.sum() + \
                    goals_scored_at[goals_scored_at.AwayTeam == rows.AwayTeam].FTAG.sum()) / 10
            else:
                home_goals = None
                away_goals = None

        direct_comparison_hg.append(home_goals)
        direct_comparison_ag.append(away_goals)

    # adding the calculated values to the data frame
    data['DirectComparisonHG'] = direct_comparison_hg
    data['DirectComparisonAG'] = direct_comparison_ag
    data['DirectComparisonGoalDiff'] = data[
            'DirectComparisonHG'] - data['DirectComparisonAG']
    data['DirectComparisonGoalQuot'] = (data['DirectComparisonHG']) / (data[
    'DirectComparisonAG'])
    return data


In [27]:
# adds the current form to the dataframe
def add_differences_on_form(df_bundesliga_with_form):
    for i in [3, 5, 10, "All"]:
        df_bundesliga_with_form[f"PDiff{i}Matches"] = df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesHT"] - df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesAT"]
        df_bundesliga_with_form[f"PQuot{i}Matches"] = df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesHT"]/df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesAT"]
    return df_bundesliga_with_form

In [28]:
# function to reduce the data frame to only the columns the model needs
def select_necessary_columns(data):
    print(data.columns)
    data_reduced = data.loc[:, [
        "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "HS", "AS",
        "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "PointsLast3MatchesHT",
        "PointsLast3MatchesAT", "PointsLast5MatchesHT", "PointsLast5MatchesAT",
        "PointsLast10MatchesHT", "PointsLast10MatchesAT", "PointsLastAllMatchesHT",
        "PointsLastAllMatchesAT", "PDiff3Matches", "PQuot3Matches", "PDiff5Matches",
        "PQuot5Matches", "PDiff10Matches", "PQuot10Matches", "PDiffAllMatches",
        "PQuotAllMatches", "MarketValueHT", "MarketValueAT", "MarketValueDiff",
        "MarketValueQuot", "HomeElo", "AwayElo", "HomeAttack", "HomeDefend", "AwayAttack",
        "AwayDefend", "HomeEloOld", "AwayEloOld", "HomeAttackOld", "HomeDefendOld",
        "AwayAttackOld", "AwayDefendOld", "DiffEloOld", "DiffAttackOld", "DiffDefendOld",
        "DiffElo", "DiffAttack", "DiffDefend", "QuotEloOld", "QuotAttackOld",
        "QuotDefendOld", "QuotElo", "QuotAttack", "QuotDefend", "DirectComparisonHG",
        "DirectComparisonAG", "DirectComparisonGoalDiff", "DirectComparisonGoalQuot"
    ]]
    return data_reduced


In [29]:
# function to save the data frame to a csv file
def save_to_csv(data_reduced):
    data_reduced.to_csv("Data\\preprocessed_dataframe_with_elo_mw_form_final_version.csv")

In [30]:
# function to run all functions with one click
def run_all(file):
    data = load_data(file)
    teams = get_all_teams(data)
    teams_elo_avstaerke = create_elo_dictionary(teams)
    data = calculate_elo_attack_defense(data, teams_elo_avstaerke)
    data = add_differences(data)
    data = add_qoutient(data)
    data = direct_comparison(data)
    data = add_differences_on_form(data)
    # print(data.columns)
    data_reduced = select_necessary_columns(data)
    # functions for the points over the last games
    # function for the market value
    save_to_csv(data_reduced)
    return data_reduced, data

file = "Data\\cumulated_data_bl_final_version.csv"
data, data_all_columns = run_all(file)
bl_dataset_06_21 = data_all_columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Div', 'Date', 'HomeTeam', 'AwayTeam',
       'FTHG', 'FTAG', 'FTR', 'HTHG',
       ...
       'DirectComparisonGoalDiff', 'DirectComparisonGoalQuot', 'PDiff3Matches',
       'PQuot3Matches', 'PDiff5Matches', 'PQuot5Matches', 'PDiff10Matches',
       'PQuot10Matches', 'PDiffAllMatches', 'PQuotAllMatches'],
      dtype='object', length=195)


# Informations about last Season
## primary question: How often wins a team?

In [31]:
# winning_team home/away
winning_team = bl_dataset_06_21['FTR'].value_counts()
print(f"Gewinnchancen \nHome Team: {winning_team['H']/winning_team.sum()}")
print(f"Away Team: {winning_team['A']/winning_team.sum()}")
print(f"Draw: {winning_team['D']/winning_team.sum()}")
# winning_team according to last matches
print(f"Gewinnchancen:")
winning_team = bl_dataset_06_21[(bl_dataset_06_21['FTR']=='H')&(bl_dataset_06_21["PDiff3Matches"]>0)|(bl_dataset_06_21['FTR']=='A')&(bl_dataset_06_21["PDiff3Matches"]<0)]
print(f"Team mit der besseren Form in den vergangenen 3 Spielen: {len(winning_team)/len(bl_dataset_06_21['PDiff3Matches'].dropna())}")
winning_team = bl_dataset_06_21[(bl_dataset_06_21['FTR']=='H')&(bl_dataset_06_21["PDiff5Matches"]>0)|(bl_dataset_06_21['FTR']=='A')&(bl_dataset_06_21["PDiff5Matches"]<0)]
print(f"Team mit der besseren Form in den vergangenen 5 Spielen: {len(winning_team)/len(bl_dataset_06_21['PDiff5Matches'].dropna())}")
winning_team = bl_dataset_06_21[(bl_dataset_06_21['FTR']=='H')&(bl_dataset_06_21["PDiff10Matches"]>0)|(bl_dataset_06_21['FTR']=='A')&(bl_dataset_06_21["PDiff10Matches"]<0)]
print(f"Team mit der besseren Form in den vergangenen 10 Spielen: {len(winning_team)/len(bl_dataset_06_21['PDiff10Matches'].dropna())}")
winning_team = bl_dataset_06_21[(bl_dataset_06_21['FTR']=='H')&(bl_dataset_06_21["PDiffAllMatches"]>0)|(bl_dataset_06_21['FTR']=='A')&(bl_dataset_06_21["PDiffAllMatches"]<0)]
print(f"Team mit der besseren Tabellenplatzierung: {len(winning_team)/len(bl_dataset_06_21['PDiffAllMatches'].dropna())}")
# winning_team according to market value
print(f"Gewinnchancen:")
winning_team = bl_dataset_06_21[(bl_dataset_06_21['FTR']=='H')&(bl_dataset_06_21["MarketValueDiff"]>0)|(bl_dataset_06_21['FTR']=='A')&(bl_dataset_06_21["MarketValueDiff"]<0)]
print(f"Team mit dem höheren Marktwert: {len(winning_team)/len(bl_dataset_06_21['MarketValueDiff'].dropna())}")
print(f"Gewinnchancen Underdog:")
winning_team = bl_dataset_06_21[(bl_dataset_06_21['FTR']=='H')&(bl_dataset_06_21["MarketValueDiff"]<0)|(bl_dataset_06_21['FTR']=='A')&(bl_dataset_06_21["MarketValueDiff"]>0)]
print(f"Team mit dem niedrigeren Marktwert: {len(winning_team)/len(bl_dataset_06_21['MarketValueDiff'].dropna())}")

Gewinnchancen 
Home Team: 0.4514161220043573
Away Team: 0.30305010893246187
Draw: 0.24553376906318083
Gewinnchancen:
Team mit der besseren Form in den vergangenen 3 Spielen: 0.37992831541218636
Team mit der besseren Form in den vergangenen 5 Spielen: 0.4192641798671436
Team mit der besseren Form in den vergangenen 10 Spielen: 0.4484567901234568
Team mit der besseren Tabellenplatzierung: 0.44814814814814813
Gewinnchancen:
Team mit dem höheren Marktwert: 0.48627450980392156
Gewinnchancen Underdog:
Team mit dem niedrigeren Marktwert: 0.2681917211328976
